In [30]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import *

/Users/captunez/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/captunez/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/captunez/anaconda/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed i

In [74]:
train = pd.read_csv('input/train1.csv', parse_dates=['visit_date'])
test = pd.read_csv('input/test1.csv', parse_dates=['visit_date'])

In [12]:
train.drop(['air_store_id', 'visit_date','id'], inplace=True, axis=1)

In [15]:
y = train['visitors']

In [75]:
train.head()

,air_store_id,visit_date,visitors,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,...,low_temperature,precipitation,hours_sunlight,solar_radiation,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover
0,air_ba937bf13d40fb24,2016-01-13,25,2,2016,1,6,0,7.0,23.843750,...,-0.766667,0.0,8.70,10.86,1.700000,4.9,1010.1,60.0,1013.1,2.5
1,air_ba937bf13d40fb24,2016-01-14,32,3,2016,1,4,0,2.0,20.292308,...,0.100000,0.0,9.05,12.09,1.966667,4.6,1008.9,52.0,1011.9,0.5
2,air_ba937bf13d40fb24,2016-01-15,29,4,2016,1,0,0,4.0,34.738462,...,2.533333,0.0,7.50,11.67,3.133333,5.7,1013.1,64.0,1016.1,6.0
3,air_ba937bf13d40fb24,2016-01-16,22,5,2016,1,2,0,6.0,27.651515,...,2.266667,0.0,8.85,12.41,2.400000,5.1,1015.8,54.0,1018.8,2.3
4,air_ba937bf13d40fb24,2016-01-18,6,0,2016,1,1,0,2.0,13.754386,...,0.333333,54.3,1.25,2.40,4.433333,7.1,995.9,95.0,998.9,7.5


In [32]:
rf = RandomForestRegressor(n_estimators=50, min_samples_leaf=10)
rf.fit(train, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [54]:
from sklearn.model_selection import cross_val_score
rf = RandomForestRegressor(n_estimators=500, min_samples_leaf=1, n_jobs=-1)
#scores = cross_val_score(rf, train[:4000], y[:4000], cv=5)

In [50]:
split = int(len(train)*0.85)
X_train, X_val = train[:split], train[split:]
y_train, y_val = y[:split], y[split:]

In [63]:
rf.fit(X_train, np.log1p(y_train))

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [64]:
RMSLE(rf.predict(X_val), np.log1p(y_val))

0.50623779331653962

In [72]:
sub_val = train[split:]
sub_val['diff'] = diff


/Users/captunez/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [71]:
diff = abs(rf.predict(X_val) - np.log1p(y_val))

In [73]:
sub_val.groupby(['dow', 'month','day_of_wee'])

,dow,year,month,day_of_week,holiday_flg,min_visitors,mean_visitors,median_visitors,max_visitors,count_observations,...,precipitation,hours_sunlight,solar_radiation,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover,diff
214291,2,2016,8,6,0,7.0,13.975610,14.0,29.0,41.0,...,6.9,10.05,24.16,5.233333,30.6,995.6,79.0,998.3,7.3,0.085312
214292,3,2016,8,4,0,7.0,13.690476,13.0,27.0,42.0,...,34.2,0.70,7.40,2.600000,32.6,1003.6,92.0,1006.3,10.0,0.437432
214293,4,2016,8,0,0,6.0,17.488372,17.0,29.0,43.0,...,4.6,6.95,19.77,4.133333,29.7,1005.1,81.0,1007.8,9.3,0.070517
214294,5,2016,8,2,0,11.0,20.837209,19.0,36.0,43.0,...,52.2,1.15,9.06,3.000000,31.0,1003.0,94.0,1005.7,9.8,0.044438
214295,6,2016,8,3,0,1.0,4.976190,4.5,16.0,42.0,...,0.1,7.80,21.00,2.800000,30.9,1003.4,83.0,1006.1,9.5,0.417345


In [65]:
X_val.shape

(37817, 63)

In [69]:
temp = []
for i in range(len(X_val)):
    temp.append(RMSLE([np.log1p(y_val.values[i])], [rf.predict(X_val.iloc[i].reshape(-1, X_val.shape[1]))]))

/Users/captunez/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  This is separate from the ipykernel package so we can avoid doing imports until


KeyboardInterrupt: 

In [22]:
def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred)**0.5

In [25]:
indexes = np.argsort(rf.feature_importances_)


In [33]:
pred = rf.predict(train)
print(RMSLE(y, pred))

8.61980281492


In [34]:
for i in indexes:
    print(train.columns[i], rf.feature_importances_[i])

air_area_name9 7.61807811617e-06
air_area_name7 6.68506036027e-06
air_area_name8 2.78944200042e-05
air_genre_name4 7.81418339149e-06
air_genre_name3 2.41636143282e-06
air_genre_name7 3.99629023742e-06
air_genre_name9 0.0
air_genre_name8 1.07260997124e-06
total_reserv_dt_diff_mean 0.0
total_reserv_mean 0.0
air_genre_name6 1.33734716327e-05
air_genre_name5 7.57517448786e-06
total_reserv_sum 0.0
rs1_y 0.0
rv1_y 4.94626893866e-06
rv2_y 4.79096279595e-06
rs2_y 0.0
air_area_name6 5.30118446213e-05
air_genre_name2 8.89396848669e-05
rs1_x 0.000207361574774
rs2_x 9.91583506049e-05
rv1_x 0.000519069291041
year 0.000223680449938
air_area_name3 0.00037673630318
air_area_name5 0.000367070653197
air_area_name1 0.000479839493864
rv2_x 0.00084959377495
holiday_flg 0.00104888016198
air_area_name0 0.000575430047411
day_of_week 0.00178661700059
dow 0.00204813206811
longitude 0.00168918432272
var_max_long 0.00169255405803
lon_plus_lat 0.00217873189231
air_genre_name 0.00185046043897
air_genre_name0 0.0018

In [1]:
import numpy as np

In [10]:
a = np.random.randint(0, 20, 10)
a

array([12,  8, 11,  0, 17, 12,  7,  5,  2, 15])

In [11]:
np.setdiff1d(np.array(np.arange(0,20)),a)

array([ 1,  3,  4,  6,  9, 10, 13, 14, 16, 18, 19])